In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U spacy

Requirement already up-to-date: spacy in /Users/kelvinkafui/opt/anaconda3/lib/python3.7/site-packages (2.2.4)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install spacy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import datetime
import spacy



from nltk.stem import PorterStemmer

import warnings; warnings.simplefilter('ignore')

In [35]:
reviews = pd.read_csv("20191226-reviews.csv", header="infer", sep=",")
                      
reviews.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [36]:
items = pd.read_csv("20191226-items.csv", header="infer",sep=",")
items.head()

,asin,brand,title,url,image,rating,reviewUrl,totalReviews,price,originalPrice
0,B0000SX2UC,NaN,Dual-Band / Tri-Mode Sprint PCS Phone w/ Voice...,https://www.amazon.com/Dual-Band-Tri-Mode-Acti...,https://m.media-amazon.com/images/I/2143EBQ210...,3.0,https://www.amazon.com/product-reviews/B0000SX2UC,14,0.00,0.0
1,B0009N5L7K,Motorola,Motorola I265 phone,https://www.amazon.com/Motorola-i265-I265-phon...,https://m.media-amazon.com/images/I/419WBAVDAR...,3.0,https://www.amazon.com/product-reviews/B0009N5L7K,7,49.95,0.0
2,B000SKTZ0S,Motorola,MOTOROLA C168i AT&T CINGULAR PREPAID GOPHONE C...,https://www.amazon.com/MOTOROLA-C168i-CINGULAR...,https://m.media-amazon.com/images/I/71b+q3ydkI...,2.7,https://www.amazon.com/product-reviews/B000SKTZ0S,22,99.99,0.0
3,B001AO4OUC,Motorola,Motorola i335 Cell Phone Boost Mobile,https://www.amazon.com/Motorola-i335-Phone-Boo...,https://m.media-amazon.com/images/I/710UO8gdT+...,3.3,https://www.amazon.com/product-reviews/B001AO4OUC,21,0.00,0.0
4,B001DCJAJG,Motorola,Motorola V365 no contract cellular phone AT&T,https://www.amazon.com/Motorola-V365-contract-...,https://m.media-amazon.com/images/I/61LYNCVrrK...,3.1,https://www.amazon.com/product-reviews/B001DCJAJG,12,149.99,0.0


In [ ]:
print('Number of rows and columns:', reviews.shape)
print('Number of rows and columns:', items.shape)

In [ ]:
# Look at the summary statistics of the data
reviews.describe().T

In [ ]:
reviews.dtypes



In [ ]:
items.describe().T

In [ ]:
items.dtypes

In [6]:
# Create a summary statistics for categorical columns
colsReviewList = [x for x in reviews.columns.to_list() if str(reviews[x].dtype)=="object"]

# Create a summary statistics for categorical columns
colsItemsList = [x for x in items.columns.to_list() if str(items[x].dtype)=="object"] 

In [8]:
# Print the list for reviews that were object data types
reviews[colsReviewList].describe().T

,count,unique,top,freq
asin,67986,720,B00F2SKPIM,981
name,67984,47323,Amazon Customer,6507
date,67986,2983,"December 11, 2019",234
title,67972,41489,Five Stars,6043
body,67965,59811,Good,303


In [9]:
items[colsItemsList].describe().T

,count,unique,top,freq
asin,720,720,B07CH2FZW5,1
brand,716,10,Samsung,346
title,720,718,"Apple iPhone 6S, 64GB, Rose Gold - For AT&T / ...",2
url,720,720,https://www.amazon.com/Huawei-Display-Factory-...,1
image,720,679,https://m.media-amazon.com/images/I/817r8IrEN5...,3
reviewUrl,720,720,https://www.amazon.com/product-reviews/B01N9TM7KG,1


In [10]:
print(reviews.isnull().sum())

asin                0
name                2
rating              0
date                0
verified            0
title              14
body               21
helpfulVotes    40771
dtype: int64


In [11]:
print(items.isnull().sum())

asin             0
brand            4
title            0
url              0
image            0
rating           0
reviewUrl        0
totalReviews     0
price            0
originalPrice    0
dtype: int64


In [ ]:
# Lets look at the distribution of helpful votes
plt.hist(x = np.log10(reviews["helpfulVotes"]), alpha= 0.5, bins=15)
plt.title("Log10 of Helpful Votes for a Review")
plt.xlabel("Log10 Helpful Votes")
plt.ylabel("Count")

In [40]:
# Now we can begin to start to extract the data from reviews to prepare for aspect based sentiment analysis
amazonReviewsList = reviews['body'].to_list()

In [41]:
amazonReviewsList

["I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Rece

In [ ]:
#amazonReviewsList[0]

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kelvinkafui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Remove stopwords
nltk_stopwords = nltk.corpus.stopwords.words('english')
len(nltk_stopwords)


179

In [16]:
# Get aspects of the reviews

#"""Load the pre-trained NLP model in spacy"""

nlp = spacy.load("en_core_web_lg")


In [37]:
def get_aspects(x):
    doc=nlp(x) ## Tokenize and extract grammatical components
    doc=[i.text for i in doc if i.text not in nltk_stopwords and i.pos_=="NOUN"] ## Remove common words and retain only nouns
    doc=list(map(lambda i: i.lower(),doc)) ## Normalize text to lower case
    doc=pd.Series(doc)
    doc=doc.value_counts().head().index.tolist() ## Get 5 most frequent nouns
    return doc

In [20]:
reviews.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


reviews.head
1. Process one collection of data and summarize information from several fields

Using NLP, lets see what aspects of the product customers are talking about in their reviews

In [38]:
#"""Apply the function to get aspects from reviews"""

print(get_aspects(amazonReviewsList[0]))
print(get_aspects(amazonReviewsList[1]))
print(get_aspects(amazonReviewsList[2]))


['phone', 'case', 'way', 'thing', 'phones']
['software', 'server', 'months', 'team', 'way']
['dialing', 'phone', 'ringtones', 'feature', 'menu']


In [23]:
get_aspects("phone")

phone


In [51]:
len(amazonReviewsList)

67986

In [55]:
#change all number types and floats to a string
reviewstring = [str(i) for i in amazonReviewsList]
sampleReviews = reviewstring[:1000]
aspects = []

for i in range(0,len(sampleReviews)):
    aspects.append(get_aspects(sampleReviews[i]))#

print(aspects[0]) 


['phone', 'case', 'way', 'thing', 'phones']


In [58]:
import requests
import json

def GetSentiment(text):
    #  write code to return dict of sentiment for text
    url = 'http://text-processing.com/api/sentiment/'
    options = { 'text': text}
    response = requests.post(url, data = options)
    sentiment = response.json()
    
    return sentiment

In [60]:
# testing
GetSentiment("amazonReviewsList")

{'probability': {'neg': 0.5095519989067516,
  'neutral': 0.5950890614040548,
  'pos': 0.4904480010932484},
 'label': 'neutral'}

In [85]:
data = amazonReviewsList
for reviews in data:
    sen = GetSentiment(reviews)
    print(sen["label"] + " : " + reviews)

neg : I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> 

neg : 1.) Slow - If you want to check your missed calls list, this phone feels compelled to inform you whether or not there are any new calls in the list...it does this for about 20 seconds. Of course, it will take you half a second to read the message...and you probably don't care about what it says. 2.) Annoying - Ever had anyone scream in your ear? That's a sensation I get from this phone on a regular basis. It has the nice ability to attach a voice tag to a phone number...a useful feature...when it works. About 80% of the time, it recognizes the number I ask for, the other 20% it lets me know that it doesn't understand...by screeching (next to my head). Want to know what that feels like with your phone? Turn the ringer up to the highest setting, hold the phone next to your ear, call your phone. Not pleasant. Add to this the fact that when you're searching one of the numerous menus on the phone (menus that have at least 5 items in them), the menu system uses up a whole screen to dis

pos : I purchased this phone for my AT&T phone replacement, even though one of the FAQ mentioned it works with AT&T SIM, in my case, it didn't. Since this is my primary phone, in stead of spending time to see the reason, I'll go with other phone, which specifically says it's "unlocked".
neutral : The phone did not come with a charger and didn't have a sims card.
pos : SERVED ME WELL AS A BACK UP PHONE.


KeyboardInterrupt: 